## Definitions

We have all the imports and the class definition we are using here. For whatever reason it was throwing an error when I had the class outside. I'll try and fix that at somepoint

The class handles most of the complexity of this program. The main part is just the loop

In [2]:
import numpy as np
import pandas as pd
import json
import requests
import time

# Key for website
micro_key = 
header = {"Ocp-Apim-Subscription-Key": micro_key}

class halo:
    def __init__(self,location=''):
        """
        This class manages data for the users as it gathers more of them
        
        """
        self.data = self.fromfile(location)
        
    def get_matches(self,seed):
        """
        Gathers match ID from a given username
        
        Parameters
            seed (String): The given username to collect data for
        
        Return:
            (List): List of ID values for games
        
        """
        # Gathering Match ID's

        # Previous matches
        match_overview = "https://www.haloapi.com/stats/hw2/players/{}/matches".format(seed)
        #Pulls last 20 matches from player
        overview = requests.get(match_overview,headers= header)
        
        # Decodes website output into dict
        temp = json.JSONDecoder().decode(overview.text)
        
        # Loops through each game and pulls out the ID for the game
        ids = list()
        for itm in temp['Results']:
            ids.append(itm["MatchId"])
        return ids
    
    def get_usernames(self, seed):
        """
        Obtains all users that recently played with the given user.
        Sorts out the ones that I've already collected and adds new ones to the
        internal variable
        
        Parameters
            seed (String): The given username to collect data for
        
        Return:
            (List): List of usernames that were added to the internal variable
        
        """
        # Gathering Match ID's
        ids = self.get_matches(seed)
        
        # Output 
        usernames = list()
        
        # Check most recent game against saved most recent game
        mask = np.where(self.data['usernames'] == seed)[0]
        savedMatch = self.data.at[mask[0],'LastMatch']
        # If match matches most recent game then skip player 
        if savedMatch == ids[0]:
            return usernames
        else:
            # Add most recent game to seed if it doesn't match currently saved
            mask = np.where(self.gamertags() == seed)[0]
            self.data.at[mask,'LastMatch'] = ids[0]
        
        
        for i,itm in enumerate(ids):
            #Get match results
            matchId = itm
            if savedMatch == itm and i!=0:
                # Add most recent game to seed if it doesn't match currently saved
                mask = np.where(self.gamertags() == seed)[0]
                self.data.at[mask[0],'LastMatch'] = ids[0]
                # End the loop
                return usernames
            match_details = "https://www.haloapi.com/stats/hw2/matches/{}".format(matchId)
            specific = requests.get(match_details,headers= header)
            r = json.JSONDecoder().decode(specific.text)

            # This will put all the players in a list
            players = r['Players']

            # Each player is assigned a key from 1-number of players
            for key in players.keys():
                # Making sure the player is a human and not a computer
                if players[key]['IsHuman']:
                    # Pulling the gamertag name out of the data
                    username = players[key]['HumanPlayerId']['Gamertag']
                    # Only add it if I haven't seen it yet.
                    # Small playerbase means you'll regularly play against the same people
                    if not((self.gamertags() == username).any()):
                        usernames.append(username)
                        self.data = self.data.append({
                            'usernames': username,
                            'LastMatch' : np.nan
                        }, ignore_index=True)
            if i==(len(ids)-1):
                # Add most recent game to seed if it doesn't match currently saved
                mask = np.where(self.gamertags() == seed)[0]
                self.data.at[mask[0],'LastMatch'] = ids[0]
            # 10 requests every 10 seconds or it'll fail        
            time.sleep(1)

        return usernames
    
    def gamertags(self,sort=''):
        """
        Outputs all the usernames that have been collected so far.
        
        Parameters
            sort (String): Which column you want the data sorted by
        
        Return:
            (List): List of usernames
        
        """
        data = self.data
        if sort != '':
            data = self.data.sort_values(by=sort,ignore_index=True,na_position='first')
        return data['usernames']
    
    def createfile(self,name):
        data = self.data.sort_values(by='usernames',ignore_index=True)
        data.to_csv(name,index=False)
        return True
    def fromfile(self,name):
        return pd.read_csv(name)

## Main

This is the main part of the program. It handles the looping part

The output for the loop is (index #) (# of names in list) (# of added names). It doesn't output on indexes that didn't add to the main list

In [3]:
halo_class = halo('user_data4.csv')

In [17]:

cum = 0
for i,itm in enumerate(halo_class.gamertags(sort="LastMatch")):
    try:
        output = halo_class.get_usernames(itm)
        cum += len(output)
        if len(output) != 0:
            # This output is mostly here to give me a sense of progress
            print(i,len(halo_class.gamertags()),len(output),cum,cum/(i+1))
    except IndexError:
        print(i)
        time.sleep(1)
    except json.decoder.JSONDecodeError:
        print(i)
        time.sleep(1)

0
1
2
3
4
5
6 77859 1 1 0.14285714285714285
7 77863 4 5 0.625
8 77866 3 8 0.8888888888888888
9 77869 3 11 1.1
10 77881 12 23 2.090909090909091
12 77883 2 25 1.9230769230769231
13 77885 2 27 1.9285714285714286
14 77888 3 30 2.0
15 77892 4 34 2.125
16 77896 4 38 2.235294117647059
17 77899 3 41 2.2777777777777777
18 77908 9 50 2.6315789473684212
19 77913 5 55 2.75
20 77915 2 57 2.7142857142857144
21 77917 2 59 2.6818181818181817
22 77929 12 71 3.0869565217391304
24 77937 8 79 3.16
25 77938 1 80 3.076923076923077
26 77942 4 84 3.111111111111111
27 77948 6 90 3.2142857142857144
28 77960 12 102 3.5172413793103448
29 77968 8 110 3.6666666666666665
30 77970 2 112 3.6129032258064515
32 77971 1 113 3.4242424242424243
33 77984 13 126 3.7058823529411766
34 77985 1 127 3.6285714285714286
35 77988 3 130 3.611111111111111
36 77993 5 135 3.6486486486486487
37 77995 2 137 3.6052631578947367
38 78004 9 146 3.7435897435897436
39 78012 8 154 3.85
40 78021 9 163 3.975609756097561
41 78024 3 166 3.952380952

285 80001 12 2143 7.493006993006993
286 80005 4 2147 7.480836236933798
289 80023 18 2165 7.4655172413793105
290 80024 1 2166 7.443298969072165
291 80029 5 2171 7.434931506849315
292 80036 7 2178 7.433447098976109
293 80040 4 2182 7.421768707482993
294 80064 24 2206 7.477966101694915
295 80093 29 2235 7.550675675675675
296 80097 4 2239 7.538720538720539
298 80103 6 2245 7.508361204013378
299 80120 17 2262 7.54
300 80144 24 2286 7.59468438538206
301 80162 18 2304 7.629139072847682
302 80168 6 2310 7.623762376237623
304 80175 7 2317 7.5967213114754095
305 80179 4 2321 7.584967320261438
306 80194 15 2336 7.609120521172638
307 80201 7 2343 7.607142857142857
308 80217 16 2359 7.634304207119741
310 80219 2 2361 7.591639871382637
313 80223 4 2365 7.531847133757962
314 80228 5 2370 7.523809523809524
315 80235 7 2377 7.522151898734177
316 80246 11 2388 7.533123028391167
317 80253 7 2395 7.531446540880503
318 80255 2 2397 7.5141065830721
319 80258 3 2400 7.5
320 80262 4 2404 7.489096573208723
321

558 82170 13 4312 7.713774597495528
559 82203 33 4345 7.758928571428571
560 82211 8 4353 7.759358288770054
562 82217 6 4359 7.7424511545293075
563 82231 14 4373 7.75354609929078
564 82239 8 4381 7.753982300884956
565 82240 1 4382 7.742049469964664
566 82261 21 4403 7.765432098765432
567 82262 1 4404 7.753521126760563
568 82265 3 4407 7.745166959578207
569 82289 24 4431 7.773684210526316
570 82295 6 4437 7.770577933450087
571 82296 1 4438 7.758741258741258
572 82297 1 4439 7.7469458987783595
573 82299 2 4441 7.7369337979094075
575 82302 3 4444 7.715277777777778
576 82305 3 4447 7.707105719237435
578 82308 3 4450 7.68566493955095
579 82321 13 4463 7.694827586206896
580 82327 6 4469 7.691910499139415
581 82328 1 4470 7.680412371134021
582 82332 4 4474 7.67409948542024
583 82339 7 4481 7.6729452054794525
584 82347 8 4489 7.673504273504274
585 82358 11 4500 7.679180887372014
586 82360 2 4502 7.669505962521295
587 82367 7 4509 7.668367346938775
588 82378 11 4520 7.674023769100169
589 82389 1

837 84118 3 6260 7.470167064439141
838 84122 4 6264 7.466030989272944
839 84128 6 6270 7.464285714285714
840 84138 10 6280 7.46730083234245
841 84145 7 6287 7.466745843230404
842 84147 2 6289 7.460260972716489
843 84149 2 6291 7.453791469194313
844 84151 2 6293 7.447337278106509
845 84156 5 6298 7.444444444444445
846 84161 5 6303 7.441558441558442
847 84164 3 6306 7.436320754716981
848 84171 7 6313 7.435806831566549
850 84176 5 6318 7.424206815511163
851 84179 3 6321 7.419014084507042
852 84181 2 6323 7.412661195779601
853 84183 2 6325 7.4063231850117095
854 84194 11 6336 7.410526315789474
855 84215 21 6357 7.4264018691588785
856 84221 6 6363 7.424737456242707
857 84223 2 6365 7.418414918414919
858 84233 10 6375 7.421420256111758
859 84246 13 6388 7.427906976744186
860 84253 7 6395 7.427409988385598
862 84268 15 6410 7.42757821552723
863 84273 5 6415 7.424768518518518
864 84289 16 6431 7.434682080924856
865 84294 5 6436 7.431870669745958
866 84297 3 6439 7.426758938869665
867 84301 4 6

1104 85910 9 8052 7.2868778280542985
1105 85923 13 8065 7.292043399638336
1106 85933 10 8075 7.294489611562782
1107 85934 1 8076 7.2888086642599275
1108 85943 9 8085 7.290351668169522
1109 85944 1 8086 7.284684684684684
1110 85949 5 8091 7.282628262826282
1111 85955 6 8097 7.281474820143885
1112 85968 13 8110 7.286612758310872
1113 85971 3 8113 7.282764811490126
1114 85978 7 8120 7.2825112107623315
1115 85982 4 8124 7.279569892473118
1116 86006 24 8148 7.294538943598925
1117 86012 6 8154 7.293381037567084
1118 86024 12 8166 7.297587131367292
1119 86031 7 8173 7.297321428571428
1120 86034 3 8176 7.293487957181088
1121 86052 18 8194 7.303030303030303
1122 86054 2 8196 7.2983081032947466
1123 86058 4 8200 7.295373665480427
1124 86069 11 8211 7.298666666666667
1125 86099 30 8241 7.318827708703375
1126 86103 4 8245 7.315882874889086
1128 86105 2 8247 7.304694419840567
1129 86106 1 8248 7.299115044247787
1130 86107 1 8249 7.293545534924846
1132 86131 24 8273 7.301853486319506
1133 86133 2 82

1370 87667 10 9809 7.154631655725748
1371 87673 6 9815 7.153790087463557
1372 87675 2 9817 7.150036416605972
1373 87686 11 9828 7.152838427947598
1374 87692 6 9834 7.152
1376 87700 8 9842 7.147421931735657
1378 87707 7 9849 7.1421319796954315
1379 87739 32 9881 7.160144927536232
1381 87746 7 9888 7.154848046309696
1382 87749 3 9891 7.151843817787419
1385 87755 6 9897 7.140692640692641
1386 87756 1 9898 7.136265320836338
1387 87761 5 9903 7.134726224783861
1389 87765 4 9907 7.127338129496403
1390 87768 3 9910 7.124370956146657
1391 87773 5 9915 7.122844827586207
1393 87781 8 9923 7.118364418938307
1394 87789 8 9931 7.118996415770609
1395 87793 4 9935 7.11676217765043
1396 87794 1 9936 7.1123836793128135
1397 87795 1 9937 7.108011444921316
1398 87802 7 9944 7.107934238741959
1399 87805 3 9947 7.105
1400 87806 1 9948 7.100642398286938
1401 87815 9 9957 7.101997146932953
1402 87819 4 9961 7.099786172487526
1403 87821 2 9963 7.096153846153846
1404 87822 1 9964 7.091814946619217
1405 87829 7

1640 89380 9 11522 7.021328458257161
1641 89386 6 11528 7.0207064555420216
1642 89388 2 11530 7.017650639074863
1643 89404 16 11546 7.023114355231144
1645 89431 27 11573 7.030984204131228
1646 89445 14 11587 7.0352155434122645
1648 89454 9 11596 7.0321406913280775
1649 89461 7 11603 7.032121212121212
1650 89470 9 11612 7.033313143549364
1651 89476 6 11618 7.032687651331719
1652 89506 30 11648 7.046581972171809
1653 89528 22 11670 7.055622732769045
1654 89529 1 11671 7.051963746223565
1656 89532 3 11674 7.045262522631261
1657 89533 1 11675 7.041616405307599
1660 89534 1 11676 7.02950030102348
1661 89544 10 11686 7.031287605294826
1662 89571 27 11713 7.043295249549008
1663 89578 7 11720 7.043269230769231
1664 89590 12 11732 7.0462462462462465
1665 89592 2 11734 7.043217286914766
1666 89593 1 11735 7.039592081583684
1669 89595 2 11737 7.028143712574851
1670 89596 1 11738 7.024536205864751
1671 89617 21 11759 7.032894736842105
1672 89621 4 11763 7.031081888822475
1673 89623 2 11765 7.02807

KeyboardInterrupt: 

In [18]:
halo_class.createfile('username_data.csv')

True

In [19]:
halo_class.data

,usernames,LastMatch
0,A 1EYEMID6ET,2f9ad8b3-b0d7-4903-b6ff-5b3e2b34e64b
1,A A Ron8451,e487c6a6-e255-4e2f-a968-d892ed2ace62
2,A BOAT O CUBANS,8ed2d203-bd33-4efb-aec5-9e8a2a79be55
3,A Bad Time5046,32f30f85-e61c-4e88-998e-74153bd940eb
4,A Banner Boy,6b271ad1-f4dc-43f0-a10b-86a37e54215f
...,...,...
90093,HardMauro III,NaN
90094,GingaNaught,NaN
90095,Brandojt,NaN
90096,Theatra1,NaN


In [13]:
# Deleted Unnecessary columns
df = halo_class.data
halo_class.data = halo_class.data.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)
halo_class.data

,Unnamed: 0.1.1,usernames,LastMatch
0,0.0,A 1EYEMID6ET,2f9ad8b3-b0d7-4903-b6ff-5b3e2b34e64b
1,1.0,A A Ron8451,e487c6a6-e255-4e2f-a968-d892ed2ace62
2,2.0,A BOAT O CUBANS,8ed2d203-bd33-4efb-aec5-9e8a2a79be55
3,3.0,A Bad Time5046,32f30f85-e61c-4e88-998e-74153bd940eb
4,4.0,A Banner Boy,6b271ad1-f4dc-43f0-a10b-86a37e54215f
...,...,...,...
77853,NaN,MasoniCannon626,NaN
77854,NaN,Killer R kid,NaN
77855,NaN,Tezcatlipoca354,NaN
77856,NaN,SbD Method,NaN
